In [10]:
import pandas as pd


places_data = pd.read_csv("final-dataset/main_dataset.csv") 
reviews_data = pd.read_csv("final-dataset/review_dataset.csv")  

user_place_reviews = pd.merge(reviews_data,places_data, on='id')

# drop review_number, lat long, address, url,status, phone, types_y, price-level,review 1-5
user_place_reviews = user_place_reviews.drop(columns=['review_number', 'latitude', 'longitude', 'address', 'url', 'status', 'phone', 'types_y', 'price-level', 'review 1', 'review 2', 'review 3', 'review 4', 'review 5'])
user_place_reviews.head()

,id,types_x,review,user_id,sentiment-score,name,primary-type,rating,rating-count
0,ChIJYcGr7GSb0S0RckePBrCWikw,"hotel, lodging","It has quite small room, and the hallway is qu...",user_1828,0.506250,Singaraja Hotel (ex- POP! Hotel Hardys Singara...,hotel,4.1,2581.0
1,ChIJZbWX6Aia0S0R0tM3h1RZ1h8,"indonesian_restaurant, restaurant, food","Surprisingly, a really good warung that’s hidd...",user_1607,0.606250,Warung Bik Juk,indonesian_restaurant,4.5,648.0
2,ChIJYyHbhgia0S0RzdjNXLmcf54,"tourist_attraction, restaurant, food","Only had a fleeting visit here, came by coach,...",user_437,0.491667,Harbour Tourist Area of Buleleng,tourist_attraction,4.3,2800.0
3,ChIJ6zf9LJCb0S0RFv3BdLl61ZY,"coffee_shop, cafe, food, store","One word, underrated! How come place like this...",user_2603,0.672338,Abuela Coffee,coffee_shop,4.9,164.0
4,ChIJ63FmGgaa0S0RWD5dfwhjGHQ,"indonesian_restaurant, restaurant, food","We came here for dinner, food was good, i like...",user_1924,0.593056,Ikan Bakar Tanjung Alam Dermaga,indonesian_restaurant,4.4,138.0


In [11]:
from sklearn.preprocessing import MultiLabelBinarizer

user_place_reviews['types_x'] = user_place_reviews['types_x'].str.split(', ')

mlb = MultiLabelBinarizer()
one_hot = mlb.fit_transform(user_place_reviews['types_x'])

one_hot_df = pd.DataFrame(one_hot, columns=mlb.classes_)

user_place_reviews = pd.concat([user_place_reviews, one_hot_df], axis=1)

user_place_reviews = user_place_reviews.drop_duplicates(subset=['user_id', 'id'])

user_place_reviews = user_place_reviews.drop(columns=['types_x'])

user_place_reviews.head()

,id,review,user_id,sentiment-score,name,primary-type,rating,rating-count,airport,american_restaurant,...,tourist_attraction,travel_agency,turkish_restaurant,university,vegan_restaurant,vegetarian_restaurant,vietnamese_restaurant,wedding_venue,wholesaler,zoo
0,ChIJYcGr7GSb0S0RckePBrCWikw,"It has quite small room, and the hallway is qu...",user_1828,0.506250,Singaraja Hotel (ex- POP! Hotel Hardys Singara...,hotel,4.1,2581.0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ChIJZbWX6Aia0S0R0tM3h1RZ1h8,"Surprisingly, a really good warung that’s hidd...",user_1607,0.606250,Warung Bik Juk,indonesian_restaurant,4.5,648.0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ChIJYyHbhgia0S0RzdjNXLmcf54,"Only had a fleeting visit here, came by coach,...",user_437,0.491667,Harbour Tourist Area of Buleleng,tourist_attraction,4.3,2800.0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,ChIJ6zf9LJCb0S0RFv3BdLl61ZY,"One word, underrated! How come place like this...",user_2603,0.672338,Abuela Coffee,coffee_shop,4.9,164.0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ChIJ63FmGgaa0S0RWD5dfwhjGHQ,"We came here for dinner, food was good, i like...",user_1924,0.593056,Ikan Bakar Tanjung Alam Dermaga,indonesian_restaurant,4.4,138.0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# List of place type columns
place_types = [
    'airport', 'american_restaurant', 'amusement_center', 'amusement_park', 'art_gallery',
    'athletic_field', 'atm', 'bakery', 'banquet_hall', 'bar', 'barbecue_restaurant',
    'barber_shop', 'beauty_salon', 'bed_and_breakfast', 'bicycle_store', 'brazilian_restaurant',
    'breakfast_restaurant', 'brunch_restaurant', 'cafe', 'campground', 'camping_cabin',
    'car_rental', 'car_wash', 'cemetery', 'child_care_agency', 'chinese_restaurant', 'church',
    'clothing_store', 'coffee_shop', 'community_center', 'consultant', 'convention_center', 'cottage',
    'cultural_center', 'dog_park', 'event_venue', 'extended_stay_hotel', 'farm', 'farmstay',
    'fast_food_restaurant', 'finance', 'fitness_center', 'florist', 'food', 'french_restaurant',
    'furniture_store', 'general_contractor', 'gift_shop', 'golf_course', 'greek_restaurant',
    'grocery_store', 'guest_house', 'gym', 'hair_care', 'hair_salon', 'hamburger_restaurant',
    'health', 'heliport', 'hiking_area', 'hindu_temple', 'historical_landmark', 'home_goods_store',
    'home_improvement_store', 'hostel', 'hotel', 'ice_cream_shop', 'indian_restaurant',
    'indonesian_restaurant', 'italian_restaurant', 'japanese_restaurant', 'jewelry_store',
    'korean_restaurant', 'lebanese_restaurant', 'library', 'liquor_store', 'lodging', 'market',
    'meal_delivery', 'meal_takeaway', 'mediterranean_restaurant', 'mexican_restaurant',
    'middle_eastern_restaurant', 'mosque', 'motel', 'movie_theater', 'museum', 'national_park',
    'night_club', 'park', 'performing_arts_theater', 'pizza_restaurant',
    'place_of_worship', 'playground', 'preschool', 'private_guest_room', 'ramen_restaurant',
    'real_estate_agency', 'resort_hotel', 'rest_stop', 'restaurant', 'sandwich_shop', 'school',
    'seafood_restaurant', 'shopping_mall', 'spa', 'spanish_restaurant', 'sporting_goods_store',
    'sports_club', 'sports_complex', 'steak_house', 'store', 'supermarket', 'sushi_restaurant',
    'swimming_pool', 'thai_restaurant', 'tourist_attraction', 'travel_agency', 'turkish_restaurant',
    'university', 'vegan_restaurant', 'vegetarian_restaurant', 'vietnamese_restaurant',
    'wedding_venue', 'wholesaler', 'zoo'
]

# Sum the place type columns for each user to get their preferences
user_preferences = user_place_reviews.groupby('user_id')[place_types].sum().reset_index()

# Normalize the preferences
user_preferences[place_types] = user_preferences[place_types].div(user_preferences[place_types].sum(axis=1), axis=0)

user_preferences


,user_id,airport,american_restaurant,amusement_center,amusement_park,art_gallery,athletic_field,atm,bakery,banquet_hall,...,tourist_attraction,travel_agency,turkish_restaurant,university,vegan_restaurant,vegetarian_restaurant,vietnamese_restaurant,wedding_venue,wholesaler,zoo
0,user_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
1,user_10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.058824,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
2,user_100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
3,user_1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.058824,0.0,...,0.058824,0.0,0.0,0.0,0.058824,0.058824,0.0,0.000000,0.058824,0.0
4,user_1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2679,user_995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.028571,0.000000,0.0
2680,user_996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.045455,0.0
2681,user_997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
2682,user_998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0


In [13]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate

# Hyperparameters
embedding_dim = 50

# User model
user_input = Input(shape=(len(place_types),), name='user_input')
user_embedding = Dense(embedding_dim, activation='relu')(user_input)

# Place model
place_input = Input(shape=(len(place_types),), name='place_input')
place_embedding = Dense(embedding_dim, activation='relu')(place_input)

# Concatenate user and place embeddings
merged = Concatenate()([user_embedding, place_embedding])
dense_1 = Dense(128, activation='relu')(merged)
dense_2 = Dense(64, activation='relu')(dense_1)
output = Dense(1, activation='sigmoid')(dense_2)

# Create and compile the model
model = Model(inputs=[user_input, place_input], outputs=output)
model.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])

print(model.summary())


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user_input (InputLayer)     [(None, 125)]                0         []                            
                                                                                                  
 place_input (InputLayer)    [(None, 125)]                0         []                            
                                                                                                  
 dense_5 (Dense)             (None, 50)                   6300      ['user_input[0][0]']          
                                                                                                  
 dense_6 (Dense)             (None, 50)                   6300      ['place_input[0][0]']         
                                                                                            

In [14]:
# Ensure user_preferences and data are correctly aligned and split the data accordingly
user_preferences = user_place_reviews.groupby('user_id')[place_types].sum().reset_index()
user_preferences[place_types] = user_preferences[place_types].div(user_preferences[place_types].sum(axis=1), axis=0)

# Merge data with user preferences to align place features with user preferences
merged_data = user_place_reviews.merge(user_preferences, on='user_id', suffixes=('', '_user'))

# Prepare user features and place features for each review
user_features = merged_data[[f'{ptype}_user' for ptype in place_types]].values
place_features = merged_data[place_types].values

# Generate labels (assuming binary relevance for simplicity)
# You need to ensure your labels are the same length as user and place features
labels = np.random.randint(2, size=(len(user_features),))

# Split into train and test sets ensuring the lengths match
user_train, user_test, place_train, place_test, y_train, y_test = train_test_split(
    user_features, place_features, labels, test_size=0.3, random_state=42
)


In [15]:
# Fit the model
history = model.fit(
    [user_train, place_train],
    y_train,
    epochs=15,
    batch_size=32,
    validation_data=([user_test, place_test], y_test)
)


Epoch 1/15
587/587 [==============================] - 8s 12ms/step - loss: 0.6932 - accuracy: 0.5029 - val_loss: 0.6932 - val_accuracy: 0.5076
Epoch 2/15
587/587 [==============================] - 6s 11ms/step - loss: 0.6932 - accuracy: 0.4993 - val_loss: 0.6931 - val_accuracy: 0.5079
Epoch 3/15
587/587 [==============================] - 6s 10ms/step - loss: 0.6931 - accuracy: 0.5058 - val_loss: 0.6931 - val_accuracy: 0.5065
Epoch 4/15
587/587 [==============================] - 6s 10ms/step - loss: 0.6930 - accuracy: 0.5042 - val_loss: 0.6931 - val_accuracy: 0.5068
Epoch 5/15
587/587 [==============================] - 6s 10ms/step - loss: 0.6930 - accuracy: 0.5039 - val_loss: 0.6931 - val_accuracy: 0.5069
Epoch 6/15
587/587 [==============================] - 6s 10ms/step - loss: 0.6929 - accuracy: 0.5110 - val_loss: 0.6931 - val_accuracy: 0.5119
Epoch 7/15
587/587 [==============================] - 6s 10ms/step - loss: 0.6929 - accuracy: 0.5088 - val_loss: 0.6931 - val_accuracy: 0.5111

In [16]:
def recommend_places(user_id, top_n=5):
    user_pref = user_preferences[user_preferences['user_id'] == user_id][place_types].values
    place_features = user_place_reviews[place_types].drop_duplicates().values
    place_ids = user_place_reviews['id'].drop_duplicates().values

    predictions = model.predict([np.repeat(user_pref, len(place_features), axis=0), place_features])
    top_indices = np.argsort(predictions[:, 0])[-top_n:][::-1]

    recommended_place_ids = place_ids[top_indices]
    
    # Ensure unique recommendations by using a set
    unique_recommendations = set(recommended_place_ids)
    
    # Retrieve details for unique recommendations
    recommended_places = user_place_reviews[user_place_reviews['id'].isin(unique_recommendations)]
    sorted_recommendations = recommended_places.sort_values(by='rating', ascending=False)
    sorted_recommendations = sorted_recommendations.drop_duplicates(subset=['name'])
    
    return sorted_recommendations[['name', 'primary-type', 'rating']]


In [18]:
user_id = 'user_10'
recommendations = recommend_places(user_id)

print("Top Recommendations for User:", user_id)
print("---------------------------------")
for index, row in recommendations.iterrows():
    print(f"Name: {row['name']}")
    print(f"Type: {row['primary-type']}")
    print(f"Rating: {row['rating']}")
    print("---------------------------------")


22/22 [==============================] - 0s 4ms/step
Top Recommendations for User: user_10
---------------------------------
Name: The Lost cafe
Type: cafe
Rating: 4.9
---------------------------------
Name: Gusto Resto & Cafe
Type: pizza_restaurant
Rating: 4.8
---------------------------------
Name: Alami Resort & Restaurant
Type: hotel
Rating: 4.7
---------------------------------
Name: REJENG BALI
Type: indonesian_restaurant
Rating: 4.6
---------------------------------
Name: D'Kailash Retreat
Type: hotel
Rating: 4.6
---------------------------------


In [19]:
model.save('user_pref_model.keras')